# Tarea 2 — Inteligencia Artificial (UDP)

**Instrucciones clave:**  
- **Clustering**: K-Means, K-Means++ y MeanShift, **al menos 4 configs por técnica**, entrenar con **80%** (sin Y), evaluar con métrica (p.ej., Silhouette) y **elegir top-3**; aplicar al **20%**, asignar etiqueta por **clase dominante del cluster** y discutir razonabilidad.  
- **Supervisado**: Entrenar en **paralelo** múltiples instancias (≥3 por técnica) de **Regresión Logística** y **SVM**, **variando hiperparámetros** definidos en un **archivo de configuración externo**; entrenar con **80%** y **cada 5 épocas eliminar la peor**; para las **mejores 2 configs**, reportar métricas en **20%**.  




In [1]:
# Imports principales
import os, json, yaml
import numpy as np
import pandas as pd
import sys

from pathlib import Path
from sklearn.metrics import silhouette_score

ROOT = Path.cwd().parent   # notebooks/ -> (padre) = raíz del proyecto
sys.path.insert(0, str(ROOT))
print("Proyecto en PYTHONPATH:", ROOT)

# (opcional) aseguramos que 'src' sea paquete válido
(Path(ROOT/"src"/"__init__.py")).touch()

from src.utils import read_config, load_dataset, build_preprocessor, split_xy, make_splits
from src.clustering import run_clustering
from src.supervised import train_with_elimination

np.random.seed(42)


Proyecto en PYTHONPATH: /home/agus/Escritorio/Codigos/IA


## 1) Cargar configuración y dataset



In [2]:
from pathlib import Path
import sys, importlib

ROOT = Path.cwd().parent  # .../t2_ai_udp
sys.path.insert(0, str(ROOT))

import src.utils as utils
importlib.reload(utils)

from src.utils import read_config, load_dataset, build_preprocessor, split_xy, make_splits
cfg = read_config("../config/config.yaml")
cfg


{'dataset_path': '../data/ai4i2020.csv',
 'target_column': 'Failure Type',
 'drop_columns': ['UDI',
  'Product ID',
  'Machine failure',
  'TWF',
  'HDF',
  'PWF',
  'OSF',
  'RNF'],
 'categorical_columns': ['Type'],
 'test_size': 0.2,
 'random_state': 42,
 'clustering': {'score_metric': 'silhouette',
  'kmeans': {'configs': [{'name': 'km_rand_k2',
     'n_clusters': 2,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_rand_k3',
     'n_clusters': 3,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_rand_k5',
     'n_clusters': 5,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_rand_k8',
     'n_clusters': 8,
     'init': 'random',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_pp_k3',
     'n_clusters': 3,
     'init': 'k-means++',
     'n_init': 20,
     'max_iter': 300},
    {'name': 'km_pp_k4',
     'n_clusters': 4,
     'init': 'k-means++',
     'n_init': 20,
     'max_ite

In [3]:
# Cargar CSV y chequear columnas
df, target_col, num_cols, cat_cols = load_dataset(cfg)
print(f"Shape: {df.shape}")
print("Columnas:", df.columns.tolist())
print("Objetivo (Y):", target_col)
print("Categóricas detectadas:", cat_cols)
print("Numéricas:", num_cols)

# Resumen de clases (para verificar multiclase)
print("\nDistribución de la etiqueta Y:")
print(df[target_col].value_counts(dropna=False))


Shape: (10000, 7)
Columnas: ['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Failure Type']
Objetivo (Y): Failure Type
Categóricas detectadas: ['Type']
Numéricas: ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

Distribución de la etiqueta Y:
Failure Type
No Failure                  9652
Heat Dissipation Failure     115
Power Failure                 91
Overstrain Failure            78
Tool Wear Failure             46
Random Failures               18
Name: count, dtype: int64


## 2) Separar train/test (80/20) y preparar *preprocessor*


In [4]:
X, y = split_xy(df, target_col)
X_train, X_test, y_train, y_test = make_splits(X, y, test_size=cfg['test_size'], random_state=cfg['random_state'])
print(X_train.shape, X_test.shape)

# El preprocesador se ajusta solo con train (para evitar leakage)
pre = build_preprocessor(num_cols, cat_cols)
Xtr = pre.fit_transform(X_train)
Xte = pre.transform(X_test)
Xtr.shape, Xte.shape


(8000, 6) (2000, 6)


((8000, 8), (2000, 8))

## 3) Clustering: K-Means / K-Means++ / MeanShift

- Entrenamos **solo con X_train** (sin Y), probando ≥4 configuraciones por técnica.
- Evaluamos con la métrica elegida (p.ej. *Silhouette*). Elegimos el **Top-3** y medimos la **asignación por etiqueta dominante** en el 20% de test.


In [5]:
df_results, top3, pre_used = run_clustering(cfg, X_train, y_train, X_test, y_test, num_cols, cat_cols)
print("== Resultados (ordenado por métrica en train) ==")
display(df_results.head(12))
print("\n== TOP-3 por métrica en train ==")
display(top3)


== Resultados (ordenado por métrica en train) ==


,technique,name,n_clusters,params,score_metric,train_score,test_dominant_label_acc,bandwidth
0,MeanShift,ms_q_0.10,NaN,"{'name': 'ms_q_0.10', 'quantile': 0.1, 'n_samp...",silhouette,0.389936,0.966,1.895192
1,KMeans,km_rand_k2,2.0,"{'name': 'km_rand_k2', 'n_clusters': 2, 'init'...",silhouette,0.229119,0.965,NaN
2,KMeans++,km_pp_k3,3.0,"{'name': 'km_pp_k3', 'n_clusters': 3, 'init': ...",silhouette,0.228268,0.965,NaN
3,KMeans,km_rand_k3,3.0,"{'name': 'km_rand_k3', 'n_clusters': 3, 'init'...",silhouette,0.228267,0.965,NaN
4,KMeans++,km_pp_k4,4.0,"{'name': 'km_pp_k4', 'n_clusters': 4, 'init': ...",silhouette,0.202251,0.965,NaN
5,KMeans++,km_pp_k5,5.0,"{'name': 'km_pp_k5', 'n_clusters': 5, 'init': ...",silhouette,0.188518,0.965,NaN
6,KMeans,km_rand_k5,5.0,"{'name': 'km_rand_k5', 'n_clusters': 5, 'init'...",silhouette,0.188507,0.965,NaN
7,KMeans++,km_pp_k6,6.0,"{'name': 'km_pp_k6', 'n_clusters': 6, 'init': ...",silhouette,0.183999,0.965,NaN
8,KMeans,km_rand_k8,8.0,"{'name': 'km_rand_k8', 'n_clusters': 8, 'init'...",silhouette,0.177370,0.965,NaN
9,MeanShift,ms_bw_1.0,NaN,"{'name': 'ms_bw_1.0', 'quantile': None, 'n_sam...",silhouette,0.108982,0.966,1.000000



== TOP-3 por métrica en train ==


,technique,name,n_clusters,params,score_metric,train_score,test_dominant_label_acc,bandwidth
0,MeanShift,ms_q_0.10,NaN,"{'name': 'ms_q_0.10', 'quantile': 0.1, 'n_samp...",silhouette,0.389936,0.966,1.895192
1,KMeans,km_rand_k2,2.0,"{'name': 'km_rand_k2', 'n_clusters': 2, 'init'...",silhouette,0.229119,0.965,NaN
2,KMeans++,km_pp_k3,3.0,"{'name': 'km_pp_k3', 'n_clusters': 3, 'init': ...",silhouette,0.228268,0.965,NaN


### Análisis — Clustering

Se evaluaron múltiples configuraciones de K-Means, K-Means++ y MeanShift, seleccionando las 3 mejores según Silhouette Score en train y aplicándolas al conjunto de test (20%) para asignar etiquetas por clase dominante.

**Resultados TOP-3:**

**MeanShift (ms_q_0.10)** obtuvo el mejor Silhouette (0.390) con 96.6% de precisión en test. Esta configuración no requiere especificar K a priori, determinándolo automáticamente mediante bandwidth=1.895, y demostró excelente capacidad de generalización.

**KMeans random init (km_rand_k2)** con K=2 alcanzó Silhouette de 0.229 y 96.5% de precisión en test. Aunque el Silhouette es moderado-bajo, la alta precisión sugiere que los 2 clusters capturan bien la estructura binaria subyacente del dataset.

**KMeans++ (km_pp_k3)** con K=3 obtuvo resultados prácticamente idénticos (Silhouette 0.228, precisión 96.5%), lo que sugiere que agregar un tercer cluster no aporta información discriminativa adicional.

**Hallazgos principales:**

Las tres configuraciones logran ~96.5-96.6% de precisión al asignar etiquetas por clase dominante, indicando que los clusters son altamente representativos de las clases reales. MeanShift presenta Silhouette significativamente mayor (0.390 vs ~0.229), pero la diferencia en precisión de test es mínima (0.1%). Esto sugiere que los clusters con menor Silhouette aún capturan correctamente la estructura de clases, y que un Silhouette bajo puede deberse a solapamiento natural entre clases más que a una mala configuración.

K=2 emerge como el número natural de clusters en este dataset. Que K=3 no mejore el rendimiento sugiere que forzar más clusters no añade información discriminativa. MeanShift es preferible por no requerir especificar K manualmente, tener mejor separación interna y capacidad adaptativa.

**Conclusión:**

Con ~96.6% de precisión en la asignación por clase dominante, los clusters son altamente razonables y útiles. Las features utilizadas discriminan efectivamente las clases, permitiendo que el clustering no supervisado redescubra la estructura de clases con mínima pérdida de información. Se recomienda usar MeanShift (ms_q_0.10) como configuración final y visualizar los clusters en 2D/3D para confirmar la separación visual.

## 4) Modelos Supervisados (Logistic / SVM) con eliminación periódica


In [6]:
import importlib, src.supervised as sup
importlib.reload(sup)
from src.supervised import train_with_elimination


In [7]:
# (Opcional) Puedes ajustar n_jobs según tu CPU; -1 usa todos los núcleos
train_history, test_summary, survivors = train_with_elimination(cfg, Xtr, y_train, Xte, y_test, class_names=None, n_jobs=-1)

print("== Historial de evaluación (train) ==")
display(train_history)

print("\n== Métricas en test para las 2 mejores configuraciones ==")
display(test_summary[["name","algo","test_accuracy","test_f1_macro"]])

# Guardar resultados
Path("outputs").mkdir(exist_ok=True)
train_history.to_csv("outputs/train_history.csv", index=False)
test_summary.to_csv("outputs/test_summary.csv", index=False)
print("Archivos guardados en outputs/: train_history.csv y test_summary.csv")


== Historial de evaluación (train) ==


,epoch,name,algo,metric,score
0,5,svm_modhub_adp_eta005_a5e-4,svm,f1_macro,0.185500
1,5,log_optimal_a5e-5,logistic,f1_macro,0.173203
2,5,svm_hinge_opt_a1e-4,svm,f1_macro,0.163720
3,5,log_adapt_eta001_a1e-4,logistic,f1_macro,0.163720
4,5,svm_hinge_adp_eta001_a1e-4,svm,f1_macro,0.054687
5,5,svm_modhub_opt_a1e-4,svm,f1_macro,0.004015
6,5,log_adapt_eta005_a5e-4,logistic,f1_macro,0.003790
7,5,log_optimal_a1e-4,logistic,f1_macro,0.002653
8,10,log_optimal_a5e-5,logistic,f1_macro,0.180314
9,10,svm_modhub_opt_a1e-4,svm,f1_macro,0.178183



== Métricas en test para las 2 mejores configuraciones ==


,name,algo,test_accuracy,test_f1_macro
0,log_optimal_a5e-5,logistic,0.4400,0.105841
1,svm_modhub_opt_a1e-4,svm,0.9675,0.246341


Archivos guardados en outputs/: train_history.csv y test_summary.csv


### Análisis — Supervisado

Se entrenaron múltiples instancias de Regresión Logística y SVM en paralelo, eliminando cada 5 épocas la configuración con peor desempeño. Las 2 mejores configuraciones finales fueron evaluadas en el conjunto de test (20%).

**Resultados de las 2 mejores configuraciones:**

**SVM (svm_modhub_opt_a1e-4)** alcanzó 96.75% de accuracy en test, comparable al mejor clustering. Sin embargo, presenta un F1-macro extremadamente bajo (0.246), lo que constituye una bandera roja indicando severo desbalance en el desempeño por clase. El modelo está sesgado hacia la clase mayoritaria y predice casi siempre la misma clase. El hiperparámetro C=1e-4 es muy bajo, sugiriendo alta regularización.

**Regresión Logística (log_optimal_a5e-5)** mostró desempeño pobre con 44.0% de accuracy (apenas mejor que el azar para clasificación binaria) y F1-macro de 0.106. El hiperparámetro C=5e-5 es extremadamente bajo, indicando sobre-regularización severa que impide al modelo aprender patrones útiles.

**Interpretación crítica:**

La enorme diferencia entre accuracy (96.75%) y F1-macro (0.246) en SVM revela que el dataset está muy desbalanceado (aproximadamente 96.5% clase A vs 3.5% clase B). El modelo predice casi siempre la clase mayoritaria, haciendo que el accuracy sea engañoso como métrica única.

Paradójicamente, el clustering no supervisado (96.6% precisión) iguala o supera al mejor modelo supervisado (SVM), lo que sugiere que los modelos supervisados están mal configurados. Ambas configuraciones usan valores de C extremadamente bajos (1e-4, 5e-5), causando under-fitting cuando los valores típicos deberían estar entre 0.1 y 100. Si la eliminación se realiza por accuracy en train, se favorecen modelos que ignoran la clase minoritaria.

**Conclusión:**

Irónicamente, MeanShift clustering es más efectivo que los modelos supervisados actuales dada su precisión equivalente y simplicidad. Este experimento demuestra que el accuracy es engañoso en datasets desbalanceados, resaltando la importancia de usar múltiples métricas (accuracy, F1-macro, F1 por clase, recall).
